In [ ]:
import streamlit as st
st.markdown('# Visualizing Dynamic tables refresh lag')

In [ ]:
# Import Required Packages
import streamlit as st
import plotly.express as px
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
# Declare required functions
@st.cache_data
def get_databases():
    databases = session.sql('show databases').collect()
    list_of_dbs = [row['name'] for row in databases]
    return list_of_dbs

@st.cache_data
def get_schemas(db_name):
    schemas = session.sql(f'show schemas in database {db_name}').collect()
    list_of_sch = [row['name'] for row in schemas]
    return list_of_sch

@st.cache_data
def get_schemas(db_name):
    schemas = session.sql(f'show schemas in database {db_name}').collect()
    list_of_sch = [row['name'] for row in schemas]
    return list_of_sch

@st.cache_data
def get_dynamic_tables(full_schema_name):
    dynamic_tables = session.sql(f'show dynamic tables in schema {full_schema_name}').collect()
    list_of_dts = [row['name'] for row in dynamic_tables]
    return list_of_dts

global selected_db, selected_sch, full_schema_name, dt_search_option, submit_button

def reset_vars():
    selected_db = selected_sch = dt_search_option = full_schema_name = ''
    submit_button = False

In [ ]:
col1, col2 = st.columns(2)
with col1:
    option=st.selectbox('Choose an option:', ['NAME', 'NAME_PREFIX'], index = 1)
with col2:
    lookback_hours=st.number_input('Hours to lookback (max value = 768):', min_value = 1, max_value = 768, value = 2)


if option == 'NAME':
    reset_vars()
    db_list = get_databases()
    col3, col4 = st.columns(2)
    with col3:
        selected_db = st.selectbox('Select a database:', db_list)
    with col4:
        sch_list = get_schemas(selected_db)
        selected_sch = st.selectbox('Select a schema:', sch_list)
        if len(sch_list) == 0:
            st.warning('No schemas found')
            st.stop()

    
    full_schema_name = selected_db + '.' + selected_sch
    dt_list = get_dynamic_tables(full_schema_name)
    if len(dt_list) == 0:
        st.warning('No Dynamic tables found')
        st.stop()


    #col5, col6 = st.columns(2)
    #with col5:
    selected_dt = st.selectbox('Select a dynamic table', dt_list)
    #with col6:
    submit_button = st.button('Submit')
        
    if submit_button:
        full_dt_name = full_schema_name + '.' + selected_dt
        dt_search_option = f'NAME => \'{full_dt_name}\''
    else:
        st.stop()
        
elif option == 'NAME_PREFIX':
    reset_vars()
    name_prefix = st.text_input('Enter a prefix:', '')
    submit_button = st.button('Submit')
    if not name_prefix:
        st.warning('Enter a prefix to continue')
        st.stop()
    elif not submit_button:
        st.stop()

    dt_search_option = f'NAME_PREFIX => \'{name_prefix}\''

if not submit_button or dt_search_option == '':
    st.stop()


In [ ]:
sql = f"""
    WITH base_data AS
    (
    SELECT
          name,
          state,
          state_code,
          state_message,
          query_id,
          data_timestamp,
          refresh_start_time,
          refresh_end_time
        FROM
          TABLE (
            INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY (
              {dt_search_option}
              , ERROR_ONLY => FALSE
              , DATA_TIMESTAMP_START => DATEADD('HOUR',-{lookback_hours},CURRENT_TIMESTAMP())
              , RESULT_LIMIT => 10000
            )
          )
    )
    ,lag_data AS 
    (
        SELECT 
            name, 
            data_timestamp, 
            refresh_end_time,
            LAG(data_timestamp) OVER (PARTITION BY name ORDER BY data_timestamp) AS previous_data_timestamp
        FROM 
            base_data
        WHERE 
            state = 'SUCCEEDED'
    )
    SELECT 
        t.name, 
        t.state, 
        t.state_code, 
        t.state_message, 
        t.data_timestamp, 
        t.refresh_start_time, 
        t.refresh_end_time,
        ld.previous_data_timestamp,
        CASE 
            WHEN t.state = 'SUCCEEDED' AND ld.previous_data_timestamp IS NOT NULL 
            THEN DATEDIFF('second', ld.previous_data_timestamp, t.refresh_end_time) 
            ELSE NULL 
        END AS refresh_lag,
        ROUND((refresh_lag/60),2) as lag_in_minutes
    FROM 
        base_data t
    LEFT JOIN 
        lag_data ld
    ON 
        t.name = ld.name 
        AND t.data_timestamp = ld.data_timestamp
    ORDER BY 
        t.name, t.data_timestamp desc;
"""
data_df = session.sql(sql).to_pandas()

In [ ]:
fig = px.line(data_df, x= 'DATA_TIMESTAMP', y= 'LAG_IN_MINUTES', color='NAME',
                title = 'Lag in Minutes Over Time by Name',
                render_mode = 'svg',
                labels = { 'DATA_TIMESTAMP': 'Timestamp', 'LAG_IN_MINUTES': 'Lag in Minutes'}
             )

st.plotly_chart(fig, use_container_width=True)